# Django Templates

## Learning Objectives

By the end of this notebook, you will be able to:

1. Configure template directories in Django settings
2. Use the Django Template Language (DTL) for dynamic content
3. Work with variables and filters in templates
4. Use template tags for logic (for, if, block, extends, include)
5. Implement template inheritance for consistent layouts
6. Configure and use static files (CSS, JS, images)
7. Pass context data from views to templates

---

## 1. Introduction to Django Templates

Templates are the "T" in MVT (Model-View-Template). They define how data is presented to the user.

A Django template is a text file (usually HTML) containing:
- **Variables**: `{{ variable }}` - Display dynamic data
- **Tags**: `{% tag %}` - Logic and control flow
- **Filters**: `{{ variable|filter }}` - Transform data
- **Comments**: `{# comment #}` - Notes for developers

## 2. Template Configuration

Templates are configured in `settings.py`:

```python
# mysite/settings.py
import os
from pathlib import Path

BASE_DIR = Path(__file__).resolve().parent.parent

TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [
            BASE_DIR / 'templates',  # Project-level templates
        ],
        'APP_DIRS': True,  # Look for templates in app directories
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]
```

### Template Directory Structure

```
mysite/
    templates/              # Project-level templates (DIRS setting)
        base.html           # Base template for all pages
        404.html            # Custom error page
    catalog/
        templates/          # App-level templates (APP_DIRS)
            catalog/        # Namespace to avoid conflicts
                book_list.html
                book_detail.html
    blog/
        templates/
            blog/
                post_list.html
```

## 3. Variables

Variables display values passed from views:

```html
<!-- Basic variable -->
<h1>{{ book.title }}</h1>
<p>By {{ book.author.first_name }} {{ book.author.last_name }}</p>

<!-- Accessing dictionary items -->
<p>{{ my_dict.key }}</p>

<!-- Accessing list items (dot notation, not brackets) -->
<p>First item: {{ my_list.0 }}</p>

<!-- Calling methods (no parentheses, no arguments allowed) -->
<p>{{ book.title.upper }}</p>
```

### View that Passes Context

```python
# catalog/views.py
from django.shortcuts import render
from .models import Book

def book_detail(request, book_id):
    book = Book.objects.get(pk=book_id)
    related_books = Book.objects.filter(author=book.author).exclude(pk=book_id)[:5]
    
    context = {
        'book': book,
        'related_books': related_books,
        'show_reviews': True,
    }
    return render(request, 'catalog/book_detail.html', context)
```

## 4. Filters

Filters transform variable output:

```html
<!-- String filters -->
{{ name|lower }}                    <!-- lowercase -->
{{ name|upper }}                    <!-- UPPERCASE -->
{{ name|title }}                    <!-- Title Case -->
{{ name|capfirst }}                 <!-- First letter cap -->
{{ text|truncatewords:30 }}         <!-- Truncate to 30 words -->
{{ text|truncatechars:100 }}        <!-- Truncate to 100 chars -->
{{ text|linebreaks }}               <!-- Convert \n to <p> tags -->
{{ text|linebreaksbr }}             <!-- Convert \n to <br> -->
{{ text|striptags }}                <!-- Remove HTML tags -->
{{ html_content|safe }}             <!-- Mark as safe HTML (use carefully!) -->

<!-- Number filters -->
{{ price|floatformat:2 }}           <!-- 19.99 -->
{{ count|add:5 }}                   <!-- Add 5 to count -->
{{ items|length }}                  <!-- List/string length -->
{{ percentage|floatformat:0 }}      <!-- Round to integer -->

<!-- Date filters -->
{{ pub_date|date:"F j, Y" }}        <!-- January 15, 2024 -->
{{ pub_date|date:"Y-m-d" }}         <!-- 2024-01-15 -->
{{ pub_date|timesince }}            <!-- 3 days ago -->
{{ event_date|timeuntil }}          <!-- 5 hours from now -->

<!-- Default values -->
{{ bio|default:"No biography available" }}
{{ count|default_if_none:0 }}

<!-- List filters -->
{{ items|join:", " }}               <!-- item1, item2, item3 -->
{{ items|first }}                   <!-- First item -->
{{ items|last }}                    <!-- Last item -->
{{ items|random }}                  <!-- Random item -->
{{ items|slice:":5" }}              <!-- First 5 items -->

<!-- Boolean filters -->
{{ is_active|yesno:"Active,Inactive" }}
{{ count|pluralize }}               <!-- s if count != 1 -->
{{ count|pluralize:"es" }}          <!-- es if count != 1 -->
{{ count|pluralize:"y,ies" }}       <!-- y/ies based on count -->
```

In [ ]:
# Common filter examples

filters = {
    'Text Formatting': {
        'lower': '{{ name|lower }} -> "john doe"',
        'upper': '{{ name|upper }} -> "JOHN DOE"',
        'title': '{{ name|title }} -> "John Doe"',
        'truncatewords': '{{ bio|truncatewords:10 }} -> first 10 words...',
        'slugify': '{{ title|slugify }} -> "my-book-title"',
    },
    'Numbers': {
        'floatformat': '{{ 3.14159|floatformat:2 }} -> "3.14"',
        'add': '{{ 5|add:3 }} -> 8',
        'divisibleby': '{% if num|divisibleby:2 %} -> True if even',
    },
    'Dates': {
        'date': '{{ pub_date|date:"M d, Y" }} -> "Jan 15, 2024"',
        'time': '{{ created|time:"H:i" }} -> "14:30"',
        'timesince': '{{ created|timesince }} -> "3 days, 2 hours"',
    },
    'Lists': {
        'length': '{{ items|length }} -> 5',
        'join': '{{ tags|join:", " }} -> "python, django, web"',
        'first/last': '{{ items|first }} -> first item',
    },
    'Safety': {
        'escape': '{{ html|escape }} -> HTML entities escaped (default)',
        'safe': '{{ html|safe }} -> HTML rendered (be careful!)',
        'striptags': '{{ html|striptags }} -> HTML tags removed',
    },
}

for category, examples in filters.items():
    print(f"\n{category}:")
    print("-" * 40)
    for name, example in examples.items():
        print(f"  {name}: {example}")

## 5. Template Tags

### if/elif/else

```html
{% if user.is_authenticated %}
    <p>Welcome, {{ user.username }}!</p>
{% else %}
    <p>Please <a href="{% url 'login' %}">log in</a>.</p>
{% endif %}

{% if book.price < 10 %}
    <span class="badge">Budget</span>
{% elif book.price < 25 %}
    <span class="badge">Regular</span>
{% else %}
    <span class="badge">Premium</span>
{% endif %}

<!-- Boolean operators -->
{% if user.is_staff and user.is_active %}
    <a href="/admin/">Admin Panel</a>
{% endif %}

{% if not book.in_stock or book.discontinued %}
    <p>Currently unavailable</p>
{% endif %}

<!-- in operator -->
{% if "python" in book.title.lower %}
    <span class="tag">Python</span>
{% endif %}
```

### for loops

```html
<!-- Basic loop -->
<ul>
{% for book in books %}
    <li>{{ book.title }} - ${{ book.price }}</li>
{% endfor %}
</ul>

<!-- Empty clause -->
<ul>
{% for book in books %}
    <li>{{ book.title }}</li>
{% empty %}
    <li>No books available.</li>
{% endfor %}
</ul>

<!-- Loop variables -->
{% for book in books %}
    {{ forloop.counter }}      <!-- 1, 2, 3, ... -->
    {{ forloop.counter0 }}     <!-- 0, 1, 2, ... -->
    {{ forloop.revcounter }}   <!-- n, n-1, ..., 1 -->
    {{ forloop.first }}        <!-- True if first iteration -->
    {{ forloop.last }}         <!-- True if last iteration -->
    {{ forloop.parentloop }}   <!-- Parent loop in nested loops -->
{% endfor %}

<!-- Example with loop variables -->
<table>
{% for book in books %}
    <tr class="{% cycle 'odd' 'even' %}">
        <td>{{ forloop.counter }}</td>
        <td>{{ book.title }}</td>
        <td>{{ book.price }}</td>
    </tr>
{% endfor %}
</table>

<!-- Looping over dictionary -->
{% for key, value in my_dict.items %}
    <p>{{ key }}: {{ value }}</p>
{% endfor %}
```

### with tag (temporary variables)

```html
{% with total=business.employees.count %}
    <p>{{ business.name }} has {{ total }} employee{{ total|pluralize }}.</p>
{% endwith %}

<!-- Multiple assignments -->
{% with alpha=1 beta=2 %}
    <p>Sum: {{ alpha|add:beta }}</p>
{% endwith %}
```

### url tag

```html
<!-- Basic URL reversal -->
<a href="{% url 'catalog:book_list' %}">All Books</a>

<!-- With positional argument -->
<a href="{% url 'catalog:book_detail' book.id %}">{{ book.title }}</a>

<!-- With keyword arguments -->
<a href="{% url 'catalog:author_books' author_id=author.id %}">View Books</a>

<!-- Store in variable -->
{% url 'catalog:book_detail' book.id as book_url %}
<a href="{{ book_url }}">{{ book.title }}</a>
```

### Other Useful Tags

```html
<!-- Comments -->
{# This is a single-line comment #}

{% comment "optional note" %}
    This is a multi-line comment.
    It won't be rendered.
{% endcomment %}

<!-- Current date/time -->
{% now "Y-m-d H:i" %}
{% now "F j, Y" as current_date %}
<p>Today is {{ current_date }}</p>

<!-- Lorem ipsum placeholder -->
{% lorem 3 p %}  <!-- 3 paragraphs -->
{% lorem 10 w %} <!-- 10 words -->

<!-- Cycle through values -->
{% for item in items %}
    <tr class="{% cycle 'row1' 'row2' %}">
        <td>{{ item }}</td>
    </tr>
{% endfor %}

<!-- First non-empty value -->
{{ subtitle|default:"" }}{% firstof subtitle title "Untitled" %}

<!-- Spaceless (remove whitespace between HTML tags) -->
{% spaceless %}
    <p>
        <a href="#">Link</a>
    </p>
{% endspaceless %}
<!-- Output: <p><a href="#">Link</a></p> -->
```

## 6. Template Inheritance

Template inheritance allows you to build a base template with common elements and extend it.

### Base Template

```html
<!-- templates/base.html -->
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}My Bookstore{% endblock %}</title>
    
    <!-- CSS -->
    {% load static %}
    <link rel="stylesheet" href="{% static 'css/style.css' %}">
    {% block extra_css %}{% endblock %}
</head>
<body>
    <header>
        <nav>
            <a href="{% url 'catalog:index' %}">Home</a>
            <a href="{% url 'catalog:book_list' %}">Books</a>
            {% if user.is_authenticated %}
                <a href="{% url 'logout' %}">Logout</a>
            {% else %}
                <a href="{% url 'login' %}">Login</a>
            {% endif %}
        </nav>
    </header>
    
    <main>
        {% block content %}
        <!-- Child templates will override this -->
        {% endblock %}
    </main>
    
    <footer>
        <p>&copy; {% now "Y" %} My Bookstore</p>
    </footer>
    
    <!-- JavaScript -->
    <script src="{% static 'js/main.js' %}"></script>
    {% block extra_js %}{% endblock %}
</body>
</html>
```

### Child Template

```html
<!-- catalog/templates/catalog/book_list.html -->
{% extends 'base.html' %}

{% block title %}Books | My Bookstore{% endblock %}

{% block content %}
<h1>Our Books</h1>

<div class="book-grid">
    {% for book in books %}
        <div class="book-card">
            <h2>{{ book.title }}</h2>
            <p>By {{ book.author }}</p>
            <p class="price">${{ book.price }}</p>
            <a href="{% url 'catalog:book_detail' book.id %}">View Details</a>
        </div>
    {% empty %}
        <p>No books available.</p>
    {% endfor %}
</div>
{% endblock %}

{% block extra_css %}
<style>
    .book-grid { display: grid; grid-template-columns: repeat(3, 1fr); }
</style>
{% endblock %}
```

### Using {{ block.super }}

To include parent block content:

```html
<!-- Parent template -->
{% block sidebar %}
    <h3>Navigation</h3>
    <ul>
        <li><a href="/">Home</a></li>
    </ul>
{% endblock %}

<!-- Child template -->
{% block sidebar %}
    {{ block.super }}  <!-- Include parent's sidebar -->
    <h3>Book Categories</h3>
    <ul>
        {% for category in categories %}
            <li>{{ category.name }}</li>
        {% endfor %}
    </ul>
{% endblock %}
```

## 7. Template Includes

Include reusable template snippets:

```html
<!-- templates/includes/book_card.html -->
<div class="book-card">
    <img src="{{ book.cover_image.url }}" alt="{{ book.title }}">
    <h3>{{ book.title }}</h3>
    <p>{{ book.author }}</p>
    <p class="price">${{ book.price }}</p>
</div>

<!-- Using the include -->
{% for book in books %}
    {% include 'includes/book_card.html' %}
{% endfor %}

<!-- Include with custom context -->
{% include 'includes/book_card.html' with book=featured_book show_price=False %}

<!-- Include with only the specified variables -->
{% include 'includes/book_card.html' with book=featured_book only %}
```

## 8. Static Files

### Configuration

```python
# mysite/settings.py

# URL prefix for static files
STATIC_URL = '/static/'

# Additional directories to look for static files
STATICFILES_DIRS = [
    BASE_DIR / 'static',  # Project-level static files
]

# Directory where collectstatic will copy files (for production)
STATIC_ROOT = BASE_DIR / 'staticfiles'
```

### Directory Structure

```
mysite/
    static/                     # Project-level static files
        css/
            style.css
        js/
            main.js
        images/
            logo.png
    catalog/
        static/                 # App-level static files
            catalog/            # Namespace
                css/
                    catalog.css
                js/
                    catalog.js
```

### Using Static Files in Templates

```html
{% load static %}

<!DOCTYPE html>
<html>
<head>
    <!-- CSS -->
    <link rel="stylesheet" href="{% static 'css/style.css' %}">
    <link rel="stylesheet" href="{% static 'catalog/css/catalog.css' %}">
</head>
<body>
    <!-- Images -->
    <img src="{% static 'images/logo.png' %}" alt="Logo">
    
    <!-- JavaScript -->
    <script src="{% static 'js/main.js' %}"></script>
</body>
</html>
```

### Collecting Static Files for Production

```bash
# Collect all static files to STATIC_ROOT
python manage.py collectstatic
```

## 9. Media Files (User Uploads)

```python
# mysite/settings.py

# URL prefix for media files
MEDIA_URL = '/media/'

# Directory where uploaded files are stored
MEDIA_ROOT = BASE_DIR / 'media'
```

```python
# mysite/urls.py (for development only)
from django.conf import settings
from django.conf.urls.static import static

urlpatterns = [
    # ... your URL patterns ...
]

# Serve media files in development
if settings.DEBUG:
    urlpatterns += static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)
```

```html
<!-- Template -->
{% if book.cover_image %}
    <img src="{{ book.cover_image.url }}" alt="{{ book.title }}">
{% else %}
    <img src="{% static 'images/no-cover.png' %}" alt="No cover">
{% endif %}
```

## 10. Context Processors

Context processors add variables to every template automatically:

```python
# catalog/context_processors.py
from .models import Category

def categories(request):
    """Add all categories to every template."""
    return {
        'all_categories': Category.objects.all()
    }

def site_settings(request):
    """Add site-wide settings to every template."""
    return {
        'site_name': 'My Bookstore',
        'contact_email': 'info@mybookstore.com',
    }
```

```python
# mysite/settings.py
TEMPLATES = [
    {
        # ...
        'OPTIONS': {
            'context_processors': [
                # Default processors
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
                # Custom processors
                'catalog.context_processors.categories',
                'catalog.context_processors.site_settings',
            ],
        },
    },
]
```

```html
<!-- Now available in every template -->
<nav>
    {% for category in all_categories %}
        <a href="{{ category.get_absolute_url }}">{{ category.name }}</a>
    {% endfor %}
</nav>

<footer>
    <p>{{ site_name }} | {{ contact_email }}</p>
</footer>
```

## 11. Custom Template Tags and Filters

Create custom tags and filters:

```
catalog/
    templatetags/           # Must be this exact name
        __init__.py
        catalog_extras.py   # Your custom tags/filters
```

```python
# catalog/templatetags/catalog_extras.py
from django import template

register = template.Library()

# Custom filter
@register.filter(name='currency')
def currency(value, symbol='$'):
    """Format number as currency."""
    try:
        return f"{symbol}{value:.2f}"
    except (ValueError, TypeError):
        return value

@register.filter
def percentage(value, total):
    """Calculate percentage."""
    try:
        return f"{(value / total) * 100:.1f}%"
    except (ZeroDivisionError, TypeError):
        return "0%"

# Simple tag
@register.simple_tag
def current_time(format_string):
    """Return current time in given format."""
    from datetime import datetime
    return datetime.now().strftime(format_string)

@register.simple_tag(takes_context=True)
def user_greeting(context):
    """Return personalized greeting."""
    request = context['request']
    if request.user.is_authenticated:
        return f"Hello, {request.user.first_name or request.user.username}!"
    return "Welcome, guest!"

# Inclusion tag (renders a template)
@register.inclusion_tag('catalog/includes/book_card.html')
def show_book(book, show_price=True):
    """Render a book card."""
    return {'book': book, 'show_price': show_price}
```

```html
<!-- Using custom tags and filters -->
{% load catalog_extras %}

<p>Price: {{ book.price|currency }}</p>
<p>Price in EUR: {{ book.price|currency:"\u20ac" }}</p>
<p>Rating: {{ book.avg_rating|percentage:5 }}</p>

<p>{% current_time "%Y-%m-%d %H:%M" %}</p>
<p>{% user_greeting %}</p>

{% show_book book %}
{% show_book book show_price=False %}
```

---

## Exercises

### Exercise 1: Create a Base Template

Create a base template (`base.html`) with:
- HTML5 doctype and structure
- A navigation bar with links
- Blocks for: title, extra_css, content, extra_js
- Footer with copyright year

<details>
<summary>Click to see solution</summary>

```html
<!-- templates/base.html -->
{% load static %}
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block title %}Bookstore{% endblock %}</title>
    <link rel="stylesheet" href="{% static 'css/style.css' %}">
    {% block extra_css %}{% endblock %}
</head>
<body>
    <header>
        <nav class="navbar">
            <a href="{% url 'home' %}" class="logo">Bookstore</a>
            <ul class="nav-links">
                <li><a href="{% url 'catalog:book_list' %}">Books</a></li>
                <li><a href="{% url 'catalog:author_list' %}">Authors</a></li>
                {% if user.is_authenticated %}
                    <li><a href="{% url 'profile' %}">{{ user.username }}</a></li>
                    <li><a href="{% url 'logout' %}">Logout</a></li>
                {% else %}
                    <li><a href="{% url 'login' %}">Login</a></li>
                    <li><a href="{% url 'register' %}">Register</a></li>
                {% endif %}
            </ul>
        </nav>
    </header>
    
    <main class="container">
        {% block content %}{% endblock %}
    </main>
    
    <footer>
        <p>&copy; {% now "Y" %} Bookstore. All rights reserved.</p>
    </footer>
    
    <script src="{% static 'js/main.js' %}"></script>
    {% block extra_js %}{% endblock %}
</body>
</html>
```

</details>

### Exercise 2: Book List Template

Create a template that extends base.html and displays a list of books with:
- Title, author, price
- "In Stock" or "Out of Stock" badge
- Link to detail page
- Alternating row colors
- Message if no books

<details>
<summary>Click to see solution</summary>

```html
<!-- catalog/templates/catalog/book_list.html -->
{% extends 'base.html' %}

{% block title %}Books | Bookstore{% endblock %}

{% block content %}
<h1>Our Books</h1>
<p>Showing {{ books|length }} book{{ books|length|pluralize }}.</p>

<table class="book-table">
    <thead>
        <tr>
            <th>#</th>
            <th>Title</th>
            <th>Author</th>
            <th>Price</th>
            <th>Status</th>
            <th>Actions</th>
        </tr>
    </thead>
    <tbody>
        {% for book in books %}
        <tr class="{% cycle 'odd' 'even' %}">
            <td>{{ forloop.counter }}</td>
            <td>{{ book.title }}</td>
            <td>{{ book.author.first_name }} {{ book.author.last_name }}</td>
            <td>${{ book.price|floatformat:2 }}</td>
            <td>
                {% if book.in_stock %}
                    <span class="badge badge-success">In Stock</span>
                {% else %}
                    <span class="badge badge-danger">Out of Stock</span>
                {% endif %}
            </td>
            <td>
                <a href="{% url 'catalog:book_detail' book.id %}">View</a>
            </td>
        </tr>
        {% empty %}
        <tr>
            <td colspan="6" class="text-center">No books available.</td>
        </tr>
        {% endfor %}
    </tbody>
</table>
{% endblock %}

{% block extra_css %}
<style>
    .book-table { width: 100%; border-collapse: collapse; }
    .book-table th, .book-table td { padding: 10px; border: 1px solid #ddd; }
    .odd { background-color: #f9f9f9; }
    .badge { padding: 3px 8px; border-radius: 4px; font-size: 12px; }
    .badge-success { background-color: #28a745; color: white; }
    .badge-danger { background-color: #dc3545; color: white; }
</style>
{% endblock %}
```

</details>

### Exercise 3: Book Detail with Includes

Create a book detail template with:
- Book information (title, author, description, price)
- A reusable review card include
- Related books section

<details>
<summary>Click to see solution</summary>

```html
<!-- catalog/templates/catalog/includes/review_card.html -->
<div class="review-card">
    <div class="review-header">
        <strong>{{ review.reviewer_name }}</strong>
        <span class="rating">{'*' * review.rating}{'.' * (5 - review.rating)}</span>
    </div>
    <p class="review-date">{{ review.created_at|date:"M d, Y" }}</p>
    <p class="review-comment">{{ review.comment }}</p>
</div>
```

```html
<!-- catalog/templates/catalog/book_detail.html -->
{% extends 'base.html' %}

{% block title %}{{ book.title }} | Bookstore{% endblock %}

{% block content %}
<article class="book-detail">
    <header>
        <h1>{{ book.title }}</h1>
        <p class="author">By 
            <a href="{% url 'catalog:author_detail' book.author.id %}">
                {{ book.author.first_name }} {{ book.author.last_name }}
            </a>
        </p>
    </header>
    
    <div class="book-info">
        <p class="price">${{ book.price|floatformat:2 }}</p>
        <p class="isbn">ISBN: {{ book.isbn }}</p>
        <p class="published">Published: {{ book.published_date|date:"F Y" }}</p>
        
        {% if book.in_stock %}
            <button class="btn btn-primary">Add to Cart</button>
        {% else %}
            <button class="btn btn-disabled" disabled>Out of Stock</button>
        {% endif %}
    </div>
    
    <div class="description">
        <h2>Description</h2>
        <p>{{ book.description|default:"No description available."|linebreaks }}</p>
    </div>
</article>

<section class="reviews">
    <h2>Reviews ({{ book.reviews.count }})</h2>
    {% for review in book.reviews.all %}
        {% include 'catalog/includes/review_card.html' with review=review %}
    {% empty %}
        <p>No reviews yet. Be the first to review this book!</p>
    {% endfor %}
</section>

{% if related_books %}
<section class="related-books">
    <h2>More by {{ book.author.last_name }}</h2>
    <div class="book-grid">
        {% for related in related_books %}
        <div class="book-card">
            <h3>{{ related.title }}</h3>
            <p>${{ related.price }}</p>
            <a href="{% url 'catalog:book_detail' related.id %}">View</a>
        </div>
        {% endfor %}
    </div>
</section>
{% endif %}
{% endblock %}
```

</details>

### Exercise 4: Custom Filter

Create a custom filter called `star_rating` that converts a numeric rating (1-5) to star characters.

<details>
<summary>Click to see solution</summary>

```python
# catalog/templatetags/catalog_extras.py
from django import template

register = template.Library()

@register.filter(name='star_rating')
def star_rating(value, max_stars=5):
    """
    Convert numeric rating to star characters.
    
    Usage: {{ rating|star_rating }} or {{ rating|star_rating:5 }}
    """
    try:
        rating = int(value)
        rating = max(0, min(rating, max_stars))  # Clamp to valid range
        filled = '\u2605' * rating  # Filled star
        empty = '\u2606' * (max_stars - rating)  # Empty star
        return filled + empty
    except (ValueError, TypeError):
        return '\u2606' * max_stars  # All empty stars on error
```

```html
{% load catalog_extras %}

<p>Rating: {{ book.average_rating|star_rating }}</p>
<!-- Output: Rating: star_filled star_filled star_filled star_filled star_empty -->
```

</details>

### Exercise 5: Pagination Template

Create a reusable pagination include that can be used with any paginated list.

<details>
<summary>Click to see solution</summary>

```html
<!-- templates/includes/pagination.html -->
{% if is_paginated %}
<nav class="pagination" aria-label="Page navigation">
    <ul class="pagination-list">
        {% if page_obj.has_previous %}
            <li>
                <a href="?page=1">&laquo; First</a>
            </li>
            <li>
                <a href="?page={{ page_obj.previous_page_number }}">&lsaquo; Previous</a>
            </li>
        {% endif %}
        
        {% for num in page_obj.paginator.page_range %}
            {% if page_obj.number == num %}
                <li class="active">
                    <span>{{ num }}</span>
                </li>
            {% elif num > page_obj.number|add:'-3' and num < page_obj.number|add:'3' %}
                <li>
                    <a href="?page={{ num }}">{{ num }}</a>
                </li>
            {% endif %}
        {% endfor %}
        
        {% if page_obj.has_next %}
            <li>
                <a href="?page={{ page_obj.next_page_number }}">Next &rsaquo;</a>
            </li>
            <li>
                <a href="?page={{ page_obj.paginator.num_pages }}">Last &raquo;</a>
            </li>
        {% endif %}
    </ul>
    
    <p class="pagination-info">
        Page {{ page_obj.number }} of {{ page_obj.paginator.num_pages }}
        ({{ page_obj.paginator.count }} item{{ page_obj.paginator.count|pluralize }})
    </p>
</nav>
{% endif %}
```

```html
<!-- Using the pagination include -->
{% extends 'base.html' %}

{% block content %}
<h1>Books</h1>

<ul>
{% for book in page_obj %}
    <li>{{ book.title }}</li>
{% endfor %}
</ul>

{% include 'includes/pagination.html' %}
{% endblock %}
```

</details>

---

## Summary

In this notebook, you learned:

- **Template configuration**: DIRS and APP_DIRS settings
- **Variables**: `{{ variable }}` displays context data
- **Filters**: `{{ variable|filter }}` transforms data
- **Tags**: `{% tag %}` for logic (if, for, block, extends, include)
- **Template inheritance**: `{% extends %}` and `{% block %}` for DRY templates
- **Includes**: `{% include %}` for reusable snippets
- **Static files**: `{% static %}` for CSS, JS, images
- **Context processors**: Add variables to all templates
- **Custom tags/filters**: Extend template functionality

## Next Steps

In the next notebook, we'll explore **Forms and the Admin Interface**, where you'll learn how to:
- Create and validate Django forms
- Handle form submissions in views
- Use ModelForms for database objects
- Set up and customize the Django admin